In [1]:
# Connecting Google Drive with Colab
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Importing Required Packages
from datasets import Dataset
import pandas as pd
import json
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch
print("[DEBUG] All libraries Imported")

[DEBUG] All libraries Imported


In [ ]:
# Load synthetic data
json_file = "/content/drive/MyDrive/WEBBIES_NOTEBOOKS/PERSONAL_RESEARCH/Assessli_Research/Modified_Data.json"
with open(json_file, "r") as f:
    data = json.load(f)

print("[DEBUG] JSON Data Loaded")

rows = []
for item in data:
    profile = item["user_profile"]
    text_input = (
        f"summarize with tone={profile['tone_preference']}, "
        f"length={profile['summary_length_preference']}, "
        f"focus={profile['focus_area']}: {item['document_text']}"
    )
    target = item["user_generated_summary"]
    rows.append({"input_text": text_input, "target_text": target})

df = pd.DataFrame(rows)
dataset = Dataset.from_pandas(df)

print("[DEBUG] Dataframe Conversion is Successfull")

In [ ]:
# Initialize tokenizer and model
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)
model_name = "t5-base" # t5-small
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
print("[DEBUG] T5-Base Model with tokenizer Loaded Successfully")

In [ ]:
# Tokenize dataset
max_input_length = 1024
max_target_length = 256

In [ ]:
# The Preprocess Function
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=max_input_length,
        truncation=True
    )
    labels = tokenizer(
        examples["target_text"],
        max_length=max_target_length,
        truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
# The Tokenized dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

In [ ]:
# Train-test split
split_dataset = tokenized_dataset.train_test_split(test_size = 0.2, seed = 42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [ ]:
# # Setting the Trainer object
# trainer = transformers.Trainer(
#     model = model,
#     train_dataset = tokenized_train_dataset,
#     args = transformers.TrainingArguments(
#         output_dir = "./BergerFineTunnedModel", # The directory where the Fine Tunned Model will be stored
#         per_device_train_batch_size = 4, # The default is 2
#         gradient_accumulation_steps = 2,
#         num_train_epochs = 3, # Increase the number of epochs if needed for better training
#         learning_rate = 5e-5,
#         max_steps = 20, # Use max steps to show the result for those steps
#         bf16 = False,
#         optim = "paged_adamw_8bit",
#         logging_dir = "./log",
#         save_strategy = "epoch",
#         save_steps = 50,
#         logging_steps = 10

# ),
#     data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm = False),
# )
# model.config.use_cache = False
# trainer.train()

# print("Trainer Loaded Successfully")

# Training Arguments
training_args = TrainingArguments(
    output_dir = "/content/drive/MyDrive/WEBBIES_NOTEBOOKS/PERSONAL_RESEARCH/Assessli_Research/lbm_model",
    save_strategy = "epoch",
    learning_rate = 3e-4,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    num_train_epochs = 50,
    weight_decay = 0.01,
    save_total_limit = 2,
    logging_dir = "./log",
    optim = "paged_adamw_8bit",
    logging_steps = 10,
    save_steps = 50,
    push_to_hub = False
)
print("[DEBUG] Training Arguements are Set")

In [ ]:
# The data collator object
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
# Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,
)

print("[DEBUG] Trainer created")

In [ ]:
# Train
trainer.train()
print("[DEBUG] Into the Training Process")

In [ ]:
# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/WEBBIES_NOTEBOOKS/PERSONAL_RESEARCH/Assessli_Research/lbm_model")
tokenizer.save_pretrained("/content/drive/MyDrive/WEBBIES_NOTEBOOKS/PERSONAL_RESEARCH/Assessli_Research/lbm_model")
print("[DEBUG] Saving the Model and Tokenizer")

print("Behaviour-conditioned summarization model saved at ./lbm_model")

In [ ]:
# Example inference
def generate_summary(doc_text, tone, length, focus):
    input_text = f"summarize with tone = {tone}, length = {length}, focus = {focus}: {doc_text}"
    input_ids = tokenizer(input_text, return_tensors = "pt", truncation = True, max_length = 1024).input_ids
    outputs = model.generate(input_ids, max_length = 256, num_beams = 4, early_stopping = True)
    return tokenizer.decode(outputs[0], skip_special_tokens = True)

In [ ]:
# Example
test_doc = "Artificial Intelligence is transforming healthcare by improving diagnosis and treatment."
print(generate_summary(test_doc, "enthusiastic", "medium", "methods"))